In [1]:
import pandas as pd
import folium
import numpy as np
import os
import googlemaps
import requests
from textwrap import shorten
from bs4 import BeautifulSoup
import re

In [2]:
# Path to your text file
file_path = 'Info for map of foreclosure lawsuits.txt'

# Initialize variables to store data
properties = []
current_property = {}

# Read and parse the data
with open(file_path, 'r') as file:
    lines = [line.strip() for line in file if line.strip()]

for line in lines:
    if not line.startswith('*'):  # Address lines now have no asterisk
        if current_property:  # Check if there's an existing property being processed
            properties.append(current_property)  # Save the current property data
            current_property = {}  # Reset for the next property
        current_property['Address'] = line  # Save the address
    elif line.startswith('*'):  # Detail lines still start with '*'
        # Remove the '* ' from the beginning and split the line at the colon
        key, value = line[1:].strip().split(': ')
        current_property[key.strip()] = value.strip()

# Don't forget to add the last property
if current_property:
    properties.append(current_property)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(properties)

# Display the DataFrame
print(df)

                                         Address  \
0             7200 S VINCENNES (buildings court)   
1                                8216 S. KING DR   
2                              8031 S SOUTH PARK   
3                                8102 S MARYLAND   
4   7758 S RACINE (37 units) - (buildings court)   
5                                 8200 S PAULINA   
6                                 2900 E 79TH ST   
7                                    7157 S YALE   
8                                 8100 S PAULINA   
9                  1626 W 79TH (buildings court)   
10                                   1505 W 79TH   
11                               7816 S PHILLIPS   

                     Purchase price by Shaunte Porter  \
0   $515,000 - bought from Big Fe LLC on March 9, ...   
1                     $1,029,000 (executed 7/23/2021)   
2                        $833,000 (executed 8/2/2021)   
3                       $950,000 (executed 5/18/2021)   
4                     $1,620,500 (exec

In [3]:
df.head(20)

,Address,Purchase price by Shaunte Porter,Shaunte Porter loan amount,Purchase price by Carlos Perez,Carlos Perez loan amount,Status of property
0,7200 S VINCENNES (buildings court),"$515,000 - bought from Big Fe LLC on March 9, ...","$777,000 - executed March 10, 2021","$1.8 million - executed on March 10, 2021",$1.26 million,"In court / ongoing foreclosure case, the build..."
1,8216 S. KING DR,"$1,029,000 (executed 7/23/2021)","$1,012,000","$1,650,000 (executed 7/29/2021)","$1,155,900",foreclosed and sold through judicial sales cor...
2,8031 S SOUTH PARK,"$833,000 (executed 8/2/2021)","$847,000",$1.45 million (executed 8/2/2021),"$1,015,000",foreclosed and sold through judicial sales cor...
3,8102 S MARYLAND,"$950,000 (executed 5/18/2021)","$942,660",$1.75 million (executed 5/19/2021),"$1,225,000",foreclosed and sold through judicial sales cor...
4,7758 S RACINE (37 units) - (buildings court),"$1,620,500 (executed 3/16/2021)",$1.8 million,"$3,050,000 (executed 3/16/2021)",$2.17 million,"In Court, underwent receivership in 2023"
5,8200 S PAULINA,"$700,000 (executed 2/3/2021)","$784,352.85",$1.4 million (executed 2/3/2021),"$980,000",foreclosed and sold through judicial sales cor...
6,2900 E 79TH ST,$1.25 million (executed 1/20/2021),"$1,281,673.25","$2,075,000 (executed 1/27/2021)","$1,452,500",foreclosed and sold through judicial sales cor...
7,7157 S YALE,"$515,000 (executed 11/16/2020)","$516,214.37","$910,000 (executed 11/16/2020)","$637,000",foreclosed and sold through judicial sales cor...
8,8100 S PAULINA,"$940,000 (12/14/2020)","$946,597.34",$2.1 million (12/15/2020),$1.47 million,foreclosed and sold through judicial sales cor...
9,1626 W 79TH (buildings court),"$620,000 (direct from Chicago title land trust...","$615,496.80",$1.1 million (11/20/2020),"$770,000","in court, permanent injunction on property - c..."


## Geocode

In [4]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [5]:
df['Address'] = df['Address'] + ' Chicago, IL'

In [6]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [8]:
df.at[2,"Address"]=("8031 S. Martin Luther King Drive")
df.at[2,"lat"]=(41.74820010669799)
df.at[2,"lon"]=(-87.61452224447761)

In [9]:
df

,Address,Purchase price by Shaunte Porter,Shaunte Porter loan amount,Purchase price by Carlos Perez,Carlos Perez loan amount,Status of property,geocoded,lat,lon
0,"7200 S VINCENNES (buildings court) Chicago, IL","$515,000 - bought from Big Fe LLC on March 9, ...","$777,000 - executed March 10, 2021","$1.8 million - executed on March 10, 2021",$1.26 million,"In court / ongoing foreclosure case, the build...","(41.7635011, -87.62913139999999)",41.763501,-87.629131
1,"8216 S. KING DR Chicago, IL","$1,029,000 (executed 7/23/2021)","$1,012,000","$1,650,000 (executed 7/29/2021)","$1,155,900",foreclosed and sold through judicial sales cor...,"(41.7450106, -87.6153059)",41.745011,-87.615306
2,8031 S. Martin Luther King Drive,"$833,000 (executed 8/2/2021)","$847,000",$1.45 million (executed 8/2/2021),"$1,015,000",foreclosed and sold through judicial sales cor...,"(41.6466122, -87.609939)",41.748200,-87.614522
3,"8102 S MARYLAND Chicago, IL","$950,000 (executed 5/18/2021)","$942,660",$1.75 million (executed 5/19/2021),"$1,225,000",foreclosed and sold through judicial sales cor...,"(41.7474327, -87.6042623)",41.747433,-87.604262
4,7758 S RACINE (37 units) - (buildings court) C...,"$1,620,500 (executed 3/16/2021)",$1.8 million,"$3,050,000 (executed 3/16/2021)",$2.17 million,"In Court, underwent receivership in 2023","(41.7525182, -87.6541677)",41.752518,-87.654168
5,"8200 S PAULINA Chicago, IL","$700,000 (executed 2/3/2021)","$784,352.85",$1.4 million (executed 2/3/2021),"$980,000",foreclosed and sold through judicial sales cor...,"(41.744708, -87.6661664)",41.744708,-87.666166
6,"2900 E 79TH ST Chicago, IL",$1.25 million (executed 1/20/2021),"$1,281,673.25","$2,075,000 (executed 1/27/2021)","$1,452,500",foreclosed and sold through judicial sales cor...,"(41.75204180000001, -87.5538227)",41.752042,-87.553823
7,"7157 S YALE Chicago, IL","$515,000 (executed 11/16/2020)","$516,214.37","$910,000 (executed 11/16/2020)","$637,000",foreclosed and sold through judicial sales cor...,"(41.7637723, -87.6306187)",41.763772,-87.630619
8,"8100 S PAULINA Chicago, IL","$940,000 (12/14/2020)","$946,597.34",$2.1 million (12/15/2020),$1.47 million,foreclosed and sold through judicial sales cor...,"(41.7464388, -87.6661721)",41.746439,-87.666172
9,"1626 W 79TH (buildings court) Chicago, IL","$620,000 (direct from Chicago title land trust...","$615,496.80",$1.1 million (11/20/2020),"$770,000","in court, permanent injunction on property - c...","(41.7504478, -87.6644676)",41.750448,-87.664468


## Map Maker

In [10]:
def popup_html(row_index, max_chars=30):
    """Generate HTML content for the popup with line breaks for long text."""
    # Define a style for the popup
    style = "min-width:300px; max-width:400px; font-size:12px;"  # Adjust max-width to your preference
    popup_content = f'<div class="popup-content" style="{style}">'

    # Columns to exclude
    exclude_columns = ['geocoded', 'lat', 'lon']

    # Dynamically add data for each column except excluded ones
    for column in [col for col in df.columns if col not in exclude_columns]:
        value = df.at[row_index, column]
        value_str = str(value).strip()
        if pd.notnull(value) and value_str != "":
            # Check if the string exceeds the maximum character length
            if len(value_str) > max_chars and column == "Address":
                # Insert a <br> tag every max_chars characters
                value_str = '<br>'.join(value_str[i:i+max_chars] for i in range(0, len(value_str), max_chars))
            popup_content += f'<strong>{column}:</strong> {value_str}<br>'
    popup_content += '</div>'
    return popup_content

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Add the custom Mapbox tile layer
folium.TileLayer(
    tiles = 'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ',
    attr = 'Mapbox',
    name = 'Streets',
    overlay = False,
    control = True,
    show = True,
    min_zoom = 11,
    max_zoom = 30
    ).add_to(m)

# Loop through the DataFrame to create markers
for i in range(len(df)):
    # Create popup HTML content
    popup_content = popup_html(i)
    
    # Create a custom icon
    icon = folium.Icon(color='red')
    
    # Create a marker with a popup
    marker = folium.CircleMarker(
        location=[df.iloc[i]['lat'], df.iloc[i]['lon']],
        popup=popup_content,
        color='red',
        width=400,
        fill=True,
        radius=7
    ).add_to(m)

m

In [11]:
m.save('index.html')

In [12]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CookCountyForeclosureMap04_16_24
